In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import re
import requests
from io import BytesIO
from PIL import Image


In [6]:

# Load the tokenizer from the saved file
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

print("Tokenizer loaded successfully!")

Tokenizer loaded successfully!


In [14]:
multimodal_model = load_model('multimodel_model.h5')

print("Model loaded successfully!")

Model loaded successfully!


In [11]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [117]:
def proprocess_image(image_path,  target_size=(224, 224)):
    try:
        response = requests.get(image_path)
        if response.status_code != 200:
            return np.zeros((target_size[0], target_size[1], 3))
        image_data = BytesIO(response.content)
        
        image = Image.open(image_data)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image = image.resize(target_size)
        image = img_to_array(image) / 255.0
    
        return image
    except Exception as e:
        return np.zeros((target_size[0], target_size[1], 3))

In [77]:
dummy_post_1 = {
    "_id": "12345",
    "image_url": "https://res.cloudinary.com/dswg5in7u/image/upload/v1749163669/1034948245380915200_wr612j.jpg",
    "content": "why don't we beat that nigga ass"
}

dummy_post_2 = {
    "_id": "123456",
    "image_url":"https://res.cloudinary.com/dswg5in7u/image/upload/v1749164207/1035252480215592966_kavagy.jpg",
    "content": "EVERYbody calling you Nigger now! https://t.co/6mguJ6KIB"
}

dummy_post_3 = {
    "_id": "123456",
    "image_url":"https://res.cloudinary.com/dswg5in7u/image/upload/v1746368159/DreamerDB/bplmtumngvu9bjr0bhgu.png",
    "content": "Beautiful girl"
}

In [125]:
# Get the image path from the dataset
# Resize image to fit model input size
img_array = proprocess_image(dummy_post_3["image_url"])
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension (1, 224, 224, 3)
img_array = preprocess_input(img_array)  # Preprocessing for ResNet50

# Preprocess the text using the tokenizer (convert text to sequence)
cleaned_text =  preprocess_text(dummy_post_3["content"])

text_sequence = tokenizer.texts_to_sequences([cleaned_text])
text_padded = pad_sequences(text_sequence, maxlen=100)  # Adjust `maxlen` based on your model's input length

# Get multimodal model prediction (for multimodal classification)
multimodal_prediction = multimodal_model.predict([img_array, text_padded])

multimodal_predicted_label = np.argmax(multimodal_prediction, axis=1)  # Multi-class classification
predicted_label = multimodal_predicted_label[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


In [126]:
prob_prediction =  [round(prob * 100, 2) for prob in multimodal_prediction[0]]

In [127]:
prob_prediction

[np.float32(3.75),
 np.float32(0.55),
 np.float32(6.01),
 np.float32(0.94),
 np.float32(7.06),
 np.float32(81.69)]

In [128]:
# Label mappings for multimodal model
multimodal_label_mapping = {
    0: "NotHate",
    1: "Racist",
    2: "Sexist",
    3: "Homophobe",
    4: "Religion",
    5: "OtherHate"
}

# Print the predicted label
print(f"Multimodal Model Prediction: {multimodal_label_mapping[predicted_label]}")

Multimodal Model Prediction: OtherHate
